# Crashes on a Street

In [1]:
import json
import pytz
import folium
from folium.features import DivIcon
from folium.plugins import HeatMap

import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from matplotlib._color_data import TABLEAU_COLORS

import os
os.chdir('..')
from scripts.crash_data_analysis import CrashDataAnalysis

In [2]:
cda = CrashDataAnalysis()
most_recent_crash = cda.most_recent_crash_timestamp()

## Find ID of street

In [3]:
query = """
select
sr.gid
, sr.lrsroute
, st_AsGeoJSON(sr.geom) as street_line
, sc.fullname
, count(distinct sc.masterid) as count_of_masterids
, row_number() over (partition by sr.gid, sr.lrsroute order by count(distinct sc.masterid) desc) 
    as fullname_priority

from street_routes sr
inner join street_centerline sc using (lrsroute)

group by 1,2,3,4
"""

street_ids = pd.read_sql(query, cda.conn)

In [4]:
streets_to_map = street_ids[street_ids.lrsroute.str.contains('18TH')].reset_index()
streets_to_map

,index,gid,lrsroute,street_line,fullname,count_of_masterids,fullname_priority
0,104,103,118TH AV,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",E 118TH AVE,5,1
1,137,136,18TH AV1,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",E 18TH AVE,23,1
2,138,137,18TH AV2,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",E 18TH AVE,5,1
3,139,138,18TH AV3,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",E 18TH AVE,1,1
4,140,139,18TH AV4,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",E 18TH AVE,1,1
5,141,140,18TH AV5,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",E 18TH AVE,6,1
6,4317,4217,18TH ST1,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",18TH ST,2,1
7,4318,4218,18TH ST2,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",18TH ST,2,1
8,4319,4219,18TH ST3,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",18TH ST,2,1
9,4320,4220,18TH ST4,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",18TH ST,14,1


In [5]:
this_map = folium.Map(prefer_canvas=True, tiles='Stamen Toner')

street_color = TABLEAU_COLORS['tab:orange']

for idx, row in streets_to_map.iterrows():
    folium.GeoJson(row.street_line, style_function=lambda x: {'color': street_color}).add_to(this_map)

    street_line_dict = json.loads(row.street_line)
    middle_point = int(np.round(len(street_line_dict['coordinates'][0]) / 2))
    
    folium.map.Marker(
        [street_line_dict['coordinates'][0][middle_point][1], street_line_dict['coordinates'][0][middle_point][0]]
        , icon=DivIcon(
            icon_size=(250,36)
            , icon_anchor=(0,0)
            , html=f'<div style="font-size: 20pt; color:{street_color}">gid: {row.gid}</div>'
        )
     ).add_to(this_map)

this_map.fit_bounds(this_map.get_bounds())
this_map

## Heatmap of Crashes on One Street

In [6]:
street_gid = 136

In [7]:
start_date = pytz.timezone('America/Denver').localize(datetime(2013,1,1))
end_date = pytz.timezone('America/Denver').localize(datetime(2023,6,1))

In [8]:
query = f"""
select
sr.gid
, sr.lrsroute
, st_length(sr.geom_denver) / 5280 as length_miles
, c.sbi
, c.fatality
, c.pedestrian_ind
, c.bicycle_ind
, c.geo_lon
, c.geo_lat
, c.reported_date
, c.reported_date at time zone 'America/Denver' as reported_date_local
, c.incident_address_corrected
, c.incident_id

from street_routes sr
inner join crashes c on st_dwithin(sr.geom_denver, c.geom_denver, 25)

where sr.gid = {street_gid}
and reported_date at time zone 'America/Denver' > '{start_date.strftime('%Y-%m-%d')}'
and reported_date at time zone 'America/Denver' < '{end_date.strftime('%Y-%m-%d')}'
and not at_freeway

order by reported_date desc
"""

street_crashes = pd.read_sql(query, cda.conn)

In [9]:
street_crashes.groupby(street_crashes.reported_date_local.dt.year).agg(
    crashes=('reported_date_local', 'count')
    , fatalities=('fatality', 'sum')
    , sbi=('sbi', 'sum')
    , pedestrian_ind=('pedestrian_ind', 'sum')
    , bicycle_ind=('bicycle_ind', 'sum')
).astype(int)

,crashes,fatalities,sbi,pedestrian_ind,bicycle_ind
reported_date_local,,,,,
2013,101,0,0,2,3
2014,104,0,1,5,1
2015,107,0,1,3,2
2016,79,0,1,1,1
2017,106,0,0,2,1
2018,124,0,1,2,1
2019,120,0,2,4,2
2020,68,0,2,5,2
2021,60,0,0,2,0


In [10]:
this_map = folium.Map(prefer_canvas=True)
HeatMap(street_crashes[['geo_lat', 'geo_lon']]).add_to(this_map)
this_map.fit_bounds(this_map.get_bounds())
this_map

In [11]:
days_in_data = (end_date - start_date).days

In [12]:
days_between_crashes = days_in_data / len(street_crashes)
days_between_crashes

3.9155509783728117

In [13]:
crashes_per_mile_per_week = (len(street_crashes) / street_crashes.iloc[0].length_miles) / (days_in_data/7)
crashes_per_mile_per_week

1.1296182581427447

In [14]:
street_crashes.sort_values(by='reported_date').tail(10)

,gid,lrsroute,length_miles,sbi,fatality,pedestrian_ind,bicycle_ind,geo_lon,geo_lat,reported_date,reported_date_local,incident_address_corrected,incident_id
9,136,18TH AV1,1.582608,False,False,0.0,0.0,-104.985507,39.744874,2023-03-21 19:01:00+00:00,2023-03-21 13:01:00.000000,E 18TH AVE / N LINCOLN SHERMAN ALY,2023147223
8,136,18TH AV1,1.582608,False,False,0.0,0.0,-104.976147,39.744844,2023-03-25 23:38:00+00:00,2023-03-25 17:38:00.000000,E 18TH AVE / N EMERSON ST,2023155147
7,136,18TH AV1,1.582608,False,False,0.0,0.0,-104.970909,39.744834,2023-03-27 20:14:00+00:00,2023-03-27 14:14:00.000000,E 18TH AVE / N LAFAYETTE ST,2023157729
6,136,18TH AV1,1.582608,False,False,0.0,0.0,-104.977538,39.744847,2023-03-31 20:44:00+00:00,2023-03-31 14:44:00.000000,N CLARKSON ST / E 18TH AVE,2023165987
5,136,18TH AV1,1.582608,False,False,0.0,0.0,-104.974778,39.744835,2023-04-14 14:18:00+00:00,2023-04-14 08:18:00.000000,PARK AVE / E 18TH AVE,2023192425
4,136,18TH AV1,1.582608,False,False,0.0,0.0,-104.974778,39.744835,2023-04-21 16:05:00+00:00,2023-04-21 10:05:00.000000,PARK AVE / E 18TH AVE,2023205777
3,136,18TH AV1,1.582608,False,False,0.0,1.0,-104.986119,39.744888,2023-04-23 21:40:00.000001+00:00,2023-04-23 15:40:00.000001,E 18TH AVE / N LINCOLN ST,2023210205
2,136,18TH AV1,1.582608,False,False,0.0,0.0,-104.976147,39.744844,2023-05-04 15:59:00+00:00,2023-05-04 09:59:00.000000,E 18TH AVE / N EMERSON ST,2023232295
1,136,18TH AV1,1.582608,False,False,0.0,0.0,-104.977538,39.744847,2023-05-06 23:43:00+00:00,2023-05-06 17:43:00.000000,E 18TH AVE / N CLARKSON ST,2023237260
0,136,18TH AV1,1.582608,False,False,0.0,0.0,-104.977538,39.744847,2023-05-08 16:28:00+00:00,2023-05-08 10:28:00.000000,E 18TH AVE / N CLARKSON ST,2023240407


In [15]:
cda.conn.dispose()

In [16]:
len(street_crashes)

971